In [1]:
import pandas as pd
import numpy as np
import joblib  ### para guardar model
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split

#Importar módulo de funciones
import sys
sys.path.insert(0, 'utils')  # Añadir el directorio 'utils' al PATH
import funciones
from importlib import reload
reload(funciones)  # Recarga el módulo

<module 'funciones' from 'c:\\recursos_humanos\\utils\\funciones.py'>

In [71]:
data_2016 = funciones.get_data('data_2016')

In [93]:
data  = data_2016[['Age', 'DistanceFromHome', 'MonthlyIncome','NumCompaniesWorked', 'PercentSalaryHike',
       'TrainingTimesLastYear', 'YearsAtCompany','EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance',
       'BusinessTravel','Department','EducationField', 'MaritalStatus']].copy()

for col in ['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance','BusinessTravel','Department','EducationField', 'MaritalStatus']:
       data[col] = data[col].astype('object')
       
data.NumCompaniesWorked = data.NumCompaniesWorked.astype('Int64')

Función para preprocesamiento

In [97]:
def preprocesamiento(data):
    
    data_num = data.select_dtypes('Int64')
    data_cat = data.select_dtypes('object')
    features_names = joblib.load("salidas/var_names.pkl").features.values

    # Asignar categorías
    EnvironmentSatisfaction = JobSatisfaction = {
        1: 'Bajo',
        2: 'Medio',
        3: 'Alto',
        4: 'Muy alto'
    }
    WorkLifeBalance = {
        1: 'Mala',
        2: 'Buena',
        3: 'Muy buena',
        4: 'La mejor'
    }

    # Reemplazar categorías
    data.EnvironmentSatisfaction.replace(EnvironmentSatisfaction, inplace=True)
    data.JobSatisfaction.replace(JobSatisfaction, inplace=True)
    data.WorkLifeBalance.replace(WorkLifeBalance, inplace=True)

    # Revisar que no hayan categorías raras que el modelo no conoce
    for col in data_cat:
        if col == 'EnvironmentSatisfaction':
            indices = data[~data[col].isin(['Bajo', 'Medio', 'Alto', 'Muy alto'])].index
        elif col == 'JobSatisfaction':
            indices = data[~data[col].isin(['Muy alto', 'Alto', 'Bajo', 'Medio'])].index
        elif col == 'WorkLifeBalance':
            indices = data[~data[col].isin(['La mejor', 'Buena', 'Mala', 'Muy buena'])].index
        elif col == 'BusinessTravel':
            indices = data[~data[col].isin(['Travel_Frequently', 'Travel_Rarely', 'Non-Travel'])].index
        elif col == 'Department':
            indices = data[~data[col].isin(['Sales', 'Research & Development', 'Human Resources'])].index
        elif col == 'EducationField':
            indices = data[~data[col].isin(['Life Sciences', 'Medical', 'Other', 'Technical Degree', 'Marketing', 'Human Resources'])].index
        elif col == 'MaritalStatus':
            indices = data[~data[col].isin(['Single', 'Married', 'Divorced'])].index
        data.drop(indices, inplace = True)

    # Definir límites de outliers
    lims_outliers = {}
    for col in data_num:
        q1 = data[col].quantile(.25)
        q3 = data[col].quantile(.75)
        iqr = q3 - q1
        lim_inf = q1 - 3 * iqr
        lim_sup = q3 + 3 * iqr
        lims_outliers[col] = (lim_inf, lim_sup)

    # Tratar outliers y nulos en numéricas
    for col in data_num:
        
        median = int(data[col].median())
        
        # Imputar outliers con la mediana
        lim_inf, lim_sup = lims_outliers[col]
        indices_outliers = data[ (data[col] < lim_inf) & (data[col] > lim_sup) ].index
        data.loc[indices_outliers, col] = median
        
        #Imputar faltantes con la mediana
        data[col].fillna(median, inplace = True)

    # Tratar faltantes en categóricas con la moda
    for col in data_cat:
        mean = data[col].mode()
        data[col].fillna(mean)

    # Obtener dummies
    data = pd.get_dummies(data)[features_names]

    return data